# Парсер для сбора данных о банках

In [2]:
import requests
import numpy as np
import datetime
import re
from bs4 import BeautifulSoup

from multiprocessing import Pool

import time
import pandas as pd

import re
import math

# Сбор данных по одной странице

In [24]:
# Функция, которая скачивает одну конкретную страницу
# page - номер страницы в списке
# Id - номер показателя 
# date1 - первая дата в табличке
# date2 - вторая дата в табличке

def getPage(page,ID,date1,date2):
    mainpage = 'http://www.banki.ru/banks/ratings/?PROPERTY_ID='+ ID +'&search[type]=name&sort_param=rating&sort_order=ASC&REGION_ID=0&date1='+ date1 +'&date2='+ date2 +'&IS_SHOW_GROUP=0&PAGEN_1='+ page
    html = pd.read_html(io = mainpage, keep_default_na = False, na_values = 'н/д', decimal = ',')
    # thousands = ''
    soup = pd.DataFrame(html[2]) 
    return(soup)

# Циклы

Всего у парсера будет три цикла. Один цикл по показателям, второй цикл вглубину таблицы, третий цикл по смене дат.

In [110]:
# Проход по конкретной странице и дате вглубину 
def Page_Deep(data1,data2,ID):

    k = 0
    i = 1
    while i < 20:
        if k == 0:
            data = getPage(str(i),ID,data1,data2)
            if data.iloc[:,3].count() == 0:
                i = i + 1
                continue
            else:
                k = 1
                i = i + 1
        cur_page_data = getPage(str(i),ID,data1,data2)
        if cur_page_data.iloc[:,3].count() == 0:
            i = i + 1
            continue
        data = pd.concat([data, cur_page_data])
        #data.append(cur_page_data)
        if len(cur_page_data) < 50:
            break
        i = i + 1
    return(data)


# Сгенерируем лист из дат, которые необходимо пробежать.
# февраль 17 - март 08  (2017-02-01 до  2008-03-01)
years = [str(item) for item in range(2010,2018)]
months = [str(item) for item in range(1,13)]
months2 = [ ]
for item in months:
    if len(item)==1:
        item = '0'+item
    months2.append(item)
dats = [ ]
for item in years:
    for jtem in months2:
        dats.append(item + '-' + jtem + '-01')



# Функция, которая выгружает по датам 108 таблиц по конкретному показателю
def One_ID_List_Date(ID):
    all_data = [ ]
    for idate in range(len(dats)-1):
        try:
            current_data = Page_Deep(dats[(idate+1)],dats[idate],ID)
            ddf=pd.DataFrame(current_data)
            ddf.columns = ['неважно','Лицензия_Банк',dats[(idate+1)],dats[idate],'неважно','неважно']
            df = pd.DataFrame()
            df['Лицензия_Банк']= ddf['Лицензия_Банк']
            df[dats[(idate+1)]]=ddf[dats[(idate+1)]]
            df[dats[idate]]=ddf[dats[idate]]
            all_data.append(df)
        except Exception:
            all_data.append('Пропуск в данных')            
    return(all_data)
# Создаётся два дата фрейма из одного пересовываются дынные в другой... Кривовато получилось.

In [26]:
# Функция, которая выгружает отдельный ID
def get_ID(ID,dats):
    k = 0
    for date in range(len(dats)-1):
        try:
            if k == 0:
                df = Page_Deep(dats[(date + 1)],dats[date],ID)
                if len(df) == 0:
                    df = pd.DataFrame()
                    continue
                else:
                    k = 1
                    df['дата'] = dats[(date + 1)]
                    df.columns = ['drop1','Лицензия_банк','ИД_' + ID,'drop2','drop3', 'drop4', 'дата']
                    df.drop(['drop1','drop2','drop3', 'drop4'], axis=1, inplace=True)   
            df_0 = Page_Deep(dats[(date + 1)],dats[date],ID)
            if len(df_0) == 0:
                df_0 = pd.DataFrame()
                continue
            df_0['дата'] = dats[(date + 1)]
            df_0.columns = ['drop1','Лицензия_банк','ИД_' + ID,'drop2','drop3', 'drop4', 'дата']
            df_0.drop(['drop1','drop2','drop3','drop4'], axis=1, inplace=True)
            df = pd.concat([df, df_0])
            #data.append(cur_page_data)
            df_0 = pd.DataFrame()
        except Exception:
            continue
    return(df)

# Загрузим каждый показатель отдельно

In [13]:
start_time = time.time() # время прохода по 1 индексу
data_id_10 = get_ID('10', dats)
data_id_10.to_csv('10' + '.csv',sep=',',header=True,index=False)
print("--- %s seconds ---" % (time.time() - start_time))  #время работы кода

--- 6699.352750778198 seconds ---


In [28]:
start_time = time.time() # время прохода по 1 индексу
data_id_30 = get_ID('30', dats)
data_id_30.to_csv('30_1' + '.csv',sep=',',header=True,index=False)
print("--- %s seconds ---" % (time.time() - start_time))  #время работы кода

--- 5825.775155305862 seconds ---


In [29]:
start_time = time.time() # время прохода по 1 индексу
data_id_40 = get_ID('40', dats)
data_id_40.to_csv('40_1' + '.csv',sep=',',header=True,index=False)
print("--- %s seconds ---" % (time.time() - start_time))  #время работы кода

--- 5897.56511926651 seconds ---


In [30]:
start_time = time.time() # время прохода по 1 индексу
data_id_50 = get_ID('50', dats)
data_id_50.to_csv('50_1' + '.csv',sep=',',header=True,index=False)
print("--- %s seconds ---" % (time.time() - start_time))  #время работы кода

--- 6016.22471165657 seconds ---


In [31]:
start_time = time.time() # время прохода по 1 индексу
data_id_60 = get_ID('60', dats)
data_id_60.to_csv('60_1' + '.csv',sep=',',header=True,index=False)
print("--- %s seconds ---" % (time.time() - start_time))  #время работы кода

--- 6166.177207946777 seconds ---


In [32]:
start_time = time.time() # время прохода по 1 индексу
data_id_500 = get_ID('500', dats)
data_id_500.to_csv('500_1' + '.csv',sep=',',header=True,index=False)
print("--- %s seconds ---" % (time.time() - start_time))  #время работы кода

--- 6025.059561729431 seconds ---


In [33]:
start_time = time.time() # время прохода по 1 индексу
data_id_1000 = get_ID('1000', dats)
data_id_1000.to_csv('1000_1' + '.csv',sep=',',header=True,index=False)
print("--- %s seconds ---" % (time.time() - start_time))  #время работы кода

--- 5822.108143091202 seconds ---


In [34]:
start_time = time.time() # время прохода по 1 индексу
data_id_1100 = get_ID('1100', dats)
data_id_1100.to_csv('1100_1' + '.csv',sep=',',header=True,index=False)
print("--- %s seconds ---" % (time.time() - start_time))  #время работы кода

--- 5098.475862264633 seconds ---


In [ ]:
start_time = time.time() # время прохода по 1 индексу
data_id_1550 = get_ID('1550', dats)
data_id_1550.to_csv('1550' + '.csv',sep=',',header=True,index=False)
print("--- %s seconds ---" % (time.time() - start_time))  #время работы кода

In [49]:
start_time = time.time() # время прохода по 1 индексу
data_id_200 = get_ID('200', dats)
data_id_200.to_csv('200' + '.csv',sep=',',header=True,index=False)
print("--- %s seconds ---" % (time.time() - start_time))  #время работы кода

UnboundLocalError: local variable 'df' referenced before assignment

In [35]:
start_time = time.time() # время прохода по 1 индексу
data_id_260 = get_ID('260', dats)
data_id_260.to_csv('260_1' + '.csv',sep=',',header=True,index=False)
print("--- %s seconds ---" % (time.time() - start_time))  #время работы кода

--- 5958.13623714447 seconds ---


In [36]:
start_time = time.time() # время прохода по 1 индексу
data_id_300 = get_ID('300', dats)
data_id_300.to_csv('300_1' + '.csv',sep=',',header=True,index=False)
print("--- %s seconds ---" % (time.time() - start_time))  #время работы кода

--- 6164.034796714783 seconds ---


In [37]:
start_time = time.time() # время прохода по 1 индексу
data_id_360 = get_ID('360', dats)
data_id_360.to_csv('360_1' + '.csv',sep=',',header=True,index=False)
print("--- %s seconds ---" % (time.time() - start_time))  #время работы кода

--- 5627.413195848465 seconds ---


In [38]:
start_time = time.time() # время прохода по 1 индексу
data_id_120 = get_ID('120', dats)
data_id_120.to_csv('120_1' + '.csv',sep=',',header=True,index=False)
print("--- %s seconds ---" % (time.time() - start_time))  #время работы кода

--- 5355.459072589874 seconds ---


In [39]:
start_time = time.time() # время прохода по 1 индексу
data_id_70 = get_ID('70', dats)
data_id_70.to_csv('70_1' + '.csv',sep=',',header=True,index=False)
print("--- %s seconds ---" % (time.time() - start_time))  #время работы кода

--- 5268.723621606827 seconds ---


In [40]:
start_time = time.time() # время прохода по 1 индексу
data_id_110 = get_ID('110', dats)
data_id_110.to_csv('110_1' + '.csv',sep=',',header=True,index=False)
print("--- %s seconds ---" % (time.time() - start_time))  #время работы кода

--- 5331.582255125046 seconds ---


In [41]:
start_time = time.time() # время прохода по 1 индексу
data_id_130 = get_ID('130', dats)
data_id_130.to_csv('130_1' + '.csv',sep=',',header=True,index=False)
print("--- %s seconds ---" % (time.time() - start_time))  #время работы кода

--- 5732.0552983284 seconds ---


In [42]:
start_time = time.time() # время прохода по 1 индексу
data_id_140 = get_ID('140', dats)
data_id_140.to_csv('140_1' + '.csv',sep=',',header=True,index=False)
print("--- %s seconds ---" % (time.time() - start_time))  #время работы кода

--- 7008.158195257187 seconds ---


In [43]:
start_time = time.time() # время прохода по 1 индексу
data_id_160 = get_ID('160', dats)
data_id_160.to_csv('160_1' + '.csv',sep=',',header=True,index=False)
print("--- %s seconds ---" % (time.time() - start_time))  #время работы кода

--- 6766.209030866623 seconds ---


In [44]:
start_time = time.time() # время прохода по 1 индексу
data_id_400 = get_ID('400', dats)
data_id_400.to_csv('400_1' + '.csv',sep=',',header=True,index=False)
print("--- %s seconds ---" % (time.time() - start_time))  #время работы кода

--- 6071.912994623184 seconds ---


In [46]:
start_time = time.time() # время прохода по 1 индексу
data_id_401 = get_ID('401', dats)
data_id_401.to_csv('401_1' + '.csv',sep=',',header=True,index=False)
print("--- %s seconds ---" % (time.time() - start_time))  #время работы кода

--- 242.64277982711792 seconds ---


# Догружу те ID, про которые забыл

In [111]:
start_time = time.time() # время прохода по 1 индексу
data_id_401 = get_ID('200', dats)
data_id_401.to_csv('200_1' + '.csv',sep=',',header=True,index=False)
print("--- %s seconds ---" % (time.time() - start_time))  #время работы кода

--- 7888.929535150528 seconds ---


+ 1100_1.csv уже загружен:)

# у этих ID есть не все даты:(

In [112]:
years = [str(item) for item in range(2015,2018)]
months = [str(item) for item in range(1,13)]
months2 = [ ]
for item in months:
    if len(item)==1:
        item = '0'+item
    months2.append(item)
dats = [ ]
for item in years:
    for jtem in months2:
        dats.append(item + '-' + jtem + '-01')
        
        
start_time = time.time() # время прохода по 1 индексу
data_id_401 = get_ID('1200', dats)
data_id_401.to_csv('1200_1' + '.csv',sep=',',header=True,index=False)
print("--- %s seconds ---" % (time.time() - start_time))  #время работы кода

--- 1951.9117465019226 seconds ---


In [113]:
start_time = time.time() # время прохода по 1 индексу
data_id_401 = get_ID('1300', dats)
data_id_401.to_csv('1300_1' + '.csv',sep=',',header=True,index=False)
print("--- %s seconds ---" % (time.time() - start_time))  #время работы кода

--- 1924.6526687145233 seconds ---


In [114]:
start_time = time.time() # время прохода по 1 индексу
data_id_401 = get_ID('1400', dats)
data_id_401.to_csv('1400_1' + '.csv',sep=',',header=True,index=False)
print("--- %s seconds ---" % (time.time() - start_time))  #время работы кода

--- 1804.0250220298767 seconds ---


In [115]:
start_time = time.time() # время прохода по 1 индексу
data_id_401 = get_ID('1500', dats)
data_id_401.to_csv('1500_1' + '.csv',sep=',',header=True,index=False)
print("--- %s seconds ---" % (time.time() - start_time))  #время работы кода

--- 1804.9547638893127 seconds ---


In [116]:
start_time = time.time() # время прохода по 1 индексу
data_id_401 = get_ID('401', dats)
data_id_401.to_csv('401_1' + '.csv',sep=',',header=True,index=False)
print("--- %s seconds ---" % (time.time() - start_time))  #время работы кода

--- 1748.1610455513 seconds ---


In [124]:
start_time = time.time() # время прохода по 1 индексу
data_id_401 = get_ID('1800', dats)
data_id_401.to_csv('1800_1' + '.csv',sep=',',header=True,index=False)
print("--- %s seconds ---" % (time.time() - start_time))  #время работы кода

--- 1707.7468400001526 seconds ---


In [125]:
start_time = time.time() # время прохода по 1 индексу
data_id_401 = get_ID('1700', dats)
data_id_401.to_csv('1700_1' + '.csv',sep=',',header=True,index=False)
print("--- %s seconds ---" % (time.time() - start_time))  #время работы кода

--- 1629.2353522777557 seconds ---


In [126]:
start_time = time.time() # время прохода по 1 индексу
data_id_401 = get_ID('1600', dats)
data_id_401.to_csv('1600_1' + '.csv',sep=',',header=True,index=False)
print("--- %s seconds ---" % (time.time() - start_time))  #время работы кода

--- 1603.0239281654358 seconds ---


# Объединим вс епоказатели в один датафрей

In [ ]:
## Данные, которые нужны для модели

Сопоставление критериев и их ID:

+ Активы нетто: na     10
+ Чистая прибыль ni    30
(excel) + Капитал (по форме 123) c_123     25
(excel) + Капитал (по форме 134) c_134     20
+ Кредитный портфель credit_portf   40
+ Просроченная задолженность в кредитном портфеле sunk_credit_portf   50
+ Вклады физических лиц retail_deposit      60
+ Средства предприятий и организаций organization_deposit   500
+ Рентабельность активов-нетто ROA    1000
+ Рентабельность капитала ROE         1100 
- + Уровень просроченной задолженности по кредитному портфелю sunk_credit_perc   1200
- + Уровень резервирования по кредитному портфелю reserv_credit_perc    1300
- + Уровень обеспечения кредитного портфеля залогом имущества zalog_credit_perc      1400
- + Валютный оборот к активам-нетто foreign_na            1500
- + Н1 H1    1600
- + Н2 H2    1700
- + Н3 H3    1800
+ Кредиты физическим лицам retail_credit    200
+ Просроченная задолженность по кредитам физическим лицам sunk_retail_credit   260
+ Кредиты предприятиям и организациям organization_credit   300
+ Просроченная задолженность по кредитам предприятиям и организациям sunk_organization_credit_perc   360
+ Выданные МБК ibl   120

In [ ]:
+ вложения в ценные бумаги:sec_tot 70 = подверженность рискам на фин рынках
+ высоколиквидные активы: liquid 110
+ вложения в акции sec 130
+ вложения в облигации bond 140
+ вложения в капиталы других организаций oth_cap 160
+ вклады физических лиц retail 400
- + вклады физических лиц оборот retail_deposit_fr 401

In [55]:
# Список из ID страничек и соответствующие им названия переменных. 
ID_list = ['10','30','25','20','40','50','60','500','1000','1100','1200','1300','1400','1500','1550',\
           '1600','1700','1800','200','260','300','360','120','70','110','130','140','160','400','401']

ID_names_list = ['na','ni','c_123','c_134','cp','LLPGL','dep_f','dep_b','ROA','ROE','LLP','IR','LCP','fr_na',\
                 'fr','H1','H2','H3','c_f','LLP_c_f','c_b','LLP_c_b','ibl','sec_tot','liquid','sec','bond','oth_cap',\
                 'retail','retail_fr']

На Мар-шаге мы скачали все необходимые нам показатели с сайта в кучу маленьких csv-табличек, которые нам теперь нужно переработать в одну большую таблицу, которую мы будем впоследствии использовать при оценки моделей. Самого по себе Reduce-шага здесь не будет. Будет Panel - шаг!

Мы хотим, чтобы в итоговой таблице оформление было таким, чтобы было удобно в R работать с панелями. Вроде как итоговая таблица должна идти вот в таком формате:



| Банк   | Год   | Показатель 1 | Показатель 2 | Показатель 3 |
|--------|-------|--------------|--------------|--------------|
| Банк 1 | Год 1 |              |              |              |
| Банк 1 | Год 2 |              |              |              |
| Банк 2 | Год 1 |              |              |              |
| Банк 2 | Год 2 |              |              |              |

In [127]:
# Для начала неплохо было бы выявить список из уникальных банков! 

# Функция, которая обхелиняет два лоиста по уникалтным названиям 
# Она делает пересечение двух множеств 
def union(a, b):
    return list(set(a) | set(b))

# Делаем список из датафреймов, которые мы подгружаем из папки. 
frames = [pd.read_csv(strtem +'.csv', header=0, sep=',', encoding="cp1251") for strtem in ['10', '1000_1','110_1','120_1','130_1', '140_1',\
                                                                                '160_1', '260_1', '30_1', '300_1', '360_1',\
                                                                               '40_1', '400_1', '50_1', '500_1', '60_1', '70_1', '1100_1',\
                                                                                '200_1','1300_1', '1400_1', '1500_1', '401_1',\
                                                                                '1800_1', '1700_1','1600_1']]

def Uniq_banks(frames):
    uniq_banks = list(frames[0]['Лицензия_банк'])
    for item in ['10', '1000_1','110_1','120_1','130_1', '140_1','160_1', '260_1', '30_1', '300_1', '360_1','40_1', '400_1', '50_1', '500_1', '60_1', '70_1', '1100_1', '200_1','1300_1', '1400_1', '1500_1', '401_1', '1800_1', '1700_1','1600_1']:
        uniq_banks = union(uniq_banks,item)
    return(uniq_banks)

uniq_banks = Uniq_banks(frames)

In [118]:
len(uniq_banks)

1176

In [287]:
# Соединим все показатели в один датафрейм
all_ok_v3 = frames[0].set_index(['лицензия', 'дата'])
    
i = 1
while i <= len(frames):
    a = pd.DataFrame(frames[i])
    all_ok_v3 = all_ok_v3.join(a.set_index(['лицензия', 'дата']), how = 'outer')
    all_ok_v3.drop_duplicates(keep = 'first', inplace =True)
    i = i + 1

IndexError: list index out of range

In [129]:
all_ok_v3.shape

(80311, 26)

In [383]:
all_ok_v3.tail(5)

,Лицензия_банк,ИД_10,дата,лицензия
93872,Частный Расчетно-Кассовый Центр лицензия № 342...,29 888,2017-12-01,0
93873,"Инновационный Расчетный Центр лицензия № 3447,...",24 400,2017-12-01,0
93874,"Инэко лицензия № 3520, Ивановская обл.",21 653,2017-12-01,0
93875,"АРБ-Инкасс лицензия № 3353, Москва и обл.",20 586,2017-12-01,0
93876,"21 Век лицензия № 3309, Санкт-Петербург и обл.",12 924,2017-12-01,0


In [131]:
all_ok_v3.to_csv('all_ok_v3' + '.csv',sep=',',header=True,index=True)

In [123]:
# Сохраним ко всему прочьему информацию о том какие банки уникальны.
un_banks = pd.DataFrame()
un_banks['Уникальные банки'] = uniq_banks
un_banks.to_csv('uniq_banks_v2.csv',sep='\t',header=True,index=False)

Отлично! Осталось только расставить метки о том отозвана лицензия у банка или нет. Скачаем для этого все необходимые данные с сайта.

## Информация о банках

In [34]:
def Bank_parser(number):
    bad_page = 'http://www.banki.ru/banks/memory/?by=PROPERTY_date&order=desc&PAGEN_1='+str(number)
    response = requests.get(bad_page)
    html = response.content
    soup = BeautifulSoup(html,"lxml")
    return(soup)


def get_cause(url):
    page = mainpage + url
    response = requests.get(page)
    html = response.content
    soup = BeautifulSoup(html,"lxml")
    cause = soup.findAll('dd',{'class':"margin-bottom-zero"})
    return(cause)

In [38]:
bankrot = {'Банк':[],'Лицензия':[], 'хреф': [], 'статус': [], 'ликвдата': [], 'причина': []}
for i in range(1,53):
    bank_page = Bank_parser(i)
    curvec = bank_page.findAll("tbody")[2].findAll('td')
    
    bank = [curvec[1+i*6].findAll('a')[0].text for i in range(int(len(curvec)/6))]
    lits = [curvec[2+i*6].text for i in range(int(len(curvec)/6))]
    href = [re.split('="|">',str(curvec[1+i*6].findAll('a')[0]))[1] for i in range(int(len(curvec)/6))]
    status = [curvec[3+i*6].text for i in range(int(len(curvec)/6))]
    likvid_data = [curvec[4+i*6].text for i in range(int(len(curvec)/6))]

    mainpage = 'http://www.banki.ru'
    cause = [get_cause(item) for item in href]  # убрал [0].text из-за ошибки
    bankrot['Банк'].extend(bank)
    bankrot['Лицензия'].extend(lits)
    bankrot['хреф'].extend(href)
    bankrot['статус'].extend(status)
    bankrot['ликвдата'].extend(likvid_data)
    bankrot['причина'].extend(cause)

In [37]:
closed_banks = pd.DataFrame(bankrot)
closed_banks.drop(['Банк', 'причина', 'статус', 'хреф'], inplace = True, axis = 1)

,Банк,Лицензия,ликвдата,причина,статус,хреф
0,Алжан,2491,02.03.2018,"[<dd class=""margin-bottom-zero"">Осуществленные...",отозв.,/banks/memory/bank/?id=10312268
1,Хованский,2145,28.02.2018,"[<dd class=""margin-bottom-zero"">Прекращение де...",ликв.,/banks/memory/bank/?id=10312354
2,Мастер-Капитал,3011,20.02.2018,"[<dd class=""margin-bottom-zero"">Деятельность О...",отозв.,/banks/memory/bank/?id=10291040
3,Уралкапиталбанк,2519,15.02.2018,"[<dd class=""margin-bottom-zero"">Проблемы в дея...",отозв.,/banks/memory/bank/?id=10283277
4,Финансово-Промышленный Капитал,3295,12.02.2018,"[<dd class=""margin-bottom-zero"">В деятельности...",отозв.,/banks/memory/bank/?id=10275239
5,Сибирский Банк Реконструкции и Развития,1284,06.02.2018,"[<dd class=""margin-bottom-zero"">Резкое ухудшен...",отозв.,/banks/memory/bank/?id=10263215
6,ПартнерКапиталБанк,635,02.02.2018,"[<dd class=""margin-bottom-zero"">Бизнес-модель ...",отозв.,/banks/memory/bank/?id=10256718
7,Расчетно-Кредитный Банк,103,02.02.2018,"[<dd class=""margin-bottom-zero"">Вследствие неэ...",отозв.,/banks/memory/bank/?id=10256723
8,Стар Альянс,3433,25.01.2018,"[<dd class=""margin-bottom-zero"">В деятельности...",отозв.,/banks/memory/bank/?id=10240410
9,АлтайБизнес-Банк,2388,19.01.2018,"[<dd class=""margin-bottom-zero"">Проблемы в дея...",отозв.,/banks/memory/bank/?id=10228131


In [518]:
closed_banks.head(5)

,Лицензия,ликвдата
0,2491,02.03.2018
1,2145,28.02.2018
2,3011,20.02.2018
3,2519,15.02.2018
4,3295,12.02.2018


In [748]:
closed_banks.shape

(2548, 3)

In [133]:
closed_banks.to_csv('closed_banks' + '.csv',sep=',',header=True, index=False)

NameError: name 'closed_banks' is not defined

In [3]:
closed_banks = pd.read_csv('closed_banks.csv', header=0, sep=',', encoding="cp1251")

In [6]:
closed_banks['ликвдата'] = pd.to_datetime(closed_banks['ликвдата'])

In [8]:
closed_banks.head(20)

,Банк,Лицензия,ликвдата,причина,статус,хреф
0,Алжан,2491,2018-02-03,"[<dd class=""margin-bottom-zero"">Осуществленные...",отозв.,/banks/memory/bank/?id=10312268
1,Хованский,2145,2018-02-28,"[<dd class=""margin-bottom-zero"">Прекращение де...",ликв.,/banks/memory/bank/?id=10312354
2,Мастер-Капитал,3011,2018-02-20,"[<dd class=""margin-bottom-zero"">Деятельность О...",отозв.,/banks/memory/bank/?id=10291040
3,Уралкапиталбанк,2519,2018-02-15,"[<dd class=""margin-bottom-zero"">Проблемы в дея...",отозв.,/banks/memory/bank/?id=10283277
4,Финансово-Промышленный Капитал,3295,2018-12-02,"[<dd class=""margin-bottom-zero"">В деятельности...",отозв.,/banks/memory/bank/?id=10275239
5,Сибирский Банк Реконструкции и Развития,1284,2018-06-02,"[<dd class=""margin-bottom-zero"">Резкое ухудшен...",отозв.,/banks/memory/bank/?id=10263215
6,ПартнерКапиталБанк,635,2018-02-02,"[<dd class=""margin-bottom-zero"">Бизнес-модель ...",отозв.,/banks/memory/bank/?id=10256718
7,Расчетно-Кредитный Банк,103,2018-02-02,"[<dd class=""margin-bottom-zero"">Вследствие неэ...",отозв.,/banks/memory/bank/?id=10256723
8,Стар Альянс,3433,2018-01-25,"[<dd class=""margin-bottom-zero"">В деятельности...",отозв.,/banks/memory/bank/?id=10240410
9,АлтайБизнес-Банк,2388,2018-01-19,"[<dd class=""margin-bottom-zero"">Проблемы в дея...",отозв.,/banks/memory/bank/?id=10228131


In [7]:
closed_banks[closed_banks['ликвдата'] >= pd.to_datetime('2010-01-01')].count()

Банк        547
Лицензия    547
ликвдата    547
причина     547
статус      547
хреф        547
dtype: int64

In [143]:
# Списики банков!
likvid_banks = closed_banks[closed_banks['статус']=='ликв.']['Банк']
otozv_banks = closed_banks[closed_banks['статус']=='отозв.']['Банк']

In [750]:
len(likvid_banks) + len(otozv_banks)

2550

In [145]:
# Причины по ликвидированным банкам: 
likvid_data = closed_banks[closed_banks['статус']=='ликв.']
otozv_data = closed_banks[closed_banks['статус']=='отозв.']

likvid_data.to_csv('likvid_data.csv',sep='\t',header=True,index=False)
otozv_data.to_csv('otozv_data.csv',sep='\t',header=True,index=False)

# ЭТАП 2

In [14]:
# Подгружаем списки из банков.
uniq_banks = list(pd.read_csv('uniq_banks.csv',sep='\t',header = 0, encoding="cp1251").iloc[:,0])
uniq_banks2 = [re.sub("\D",'',item) for item in uniq_banks]

print(len(uniq_banks))

# Подгружаем данные по всякой отозванной фигне. 
likvid_data = pd.read_csv('likvid_data.csv',sep='\t',header=0, encoding="cp1251")
otozv_data = pd.read_csv('otozv_data.csv',sep='\t',header=0, encoding="cp1251")
print(likvid_data.shape[0])
print(otozv_data.shape[0])

# Подгружаем табличку с данными.
bank_frame = pd.read_csv('all_ok_v3.csv',sep=',',header=0, encoding="cp1251")
#uniq_banks = set(bank_frame['Банк'])

1181
503
2047


C:\Users\imaksimov002\AppData\Local\Continuum\anaconda\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Банк,Лицензия,ликвдата,причина,статус,хреф
0,Алжан,2491,02.03.2018,"[<dd class=""margin-bottom-zero"">Осуществленные...",отозв.,/banks/memory/bank/?id=10312268
1,Мастер-Капитал,3011,20.02.2018,"[<dd class=""margin-bottom-zero"">Деятельность О...",отозв.,/banks/memory/bank/?id=10291040


In [318]:
bank_frame = bank_frame.iloc[:80287, :]

In [148]:
# Подгружаем списки из банков.
uniq_banks = list(pd.read_csv('uniq_banks_v2.csv',sep='\t',header = 0, encoding="cp1251").iloc[:,0])
uniq_banks2 = [re.sub("\D",'',item) for item in uniq_banks]

# Получим номер лицензии

In [300]:
# функция для выделения номера лицензии из строки
def get_license(a):
    license = a.split('№ ')[1].split(',')[0]
    return(license)

In [320]:
bank_frame['лицензия'] = 0
p = np.array(range(bank_frame.shape[0]))
for i in range(80287):
    p[i] = get_license(bank_frame.iloc[i, 0])
bank_frame['лицензия'] = p

In [324]:
bank_frame.to_csv('all_ok_v5' + '.csv',sep=',',header=True, index=False)

# Норматив ликвидности банка Н3 

In [508]:
liquidity = pd.read_csv('liquidity_2.csv',sep=',', header = 0,  encoding="cp1251")

In [509]:
liquidity.dropna(axis=0, how='any', inplace = True)
liquidity.columns = ['drop', 'лицензия', 'N3', 'дата']
liquidity[['лицензия']] = liquidity[['лицензия']].astype(int)
liquidity['дата'] = pd.to_datetime(liquidity['дата'])

In [510]:
liquidity.drop(['drop'], inplace = True, axis = 1)

In [511]:
liquidity.head(5)

,лицензия,N3,дата
0,1,73.16,2010-06-01
1,67,63.10,2010-06-01
2,85,90.30,2010-06-01
3,96,143.33,2010-06-01
4,101,147.25,2010-06-01


# Обработка данных по закрытым банкам

In [523]:
#closed_banks[['Лицензия']] = closed_banks[['Лицензия']].astype(int)
closed_banks['ликвдата'] = pd.to_datetime(closed_banks['ликвдата'])

In [530]:
# Данные скачались немного криво: надо поменять день и месяц местами
closed_banks['дата'] = 0
for u in range(closed_banks.shape[0] - 1):
    y = closed_banks.iloc[u, 1].year
    m = closed_banks.iloc[u, 1].month
    if len(str(m)) == 1:
        m = '-0'+ str(m)
    else:
        m = '-' + str(m)
    d = '-01'
    date = str(y) + m + d
    date = pd.to_datetime(date)
    closed_banks.iloc[u, 2] = date

In [532]:
closed_banks.drop('ликвдата', inplace = True, axis = 1)

In [534]:
closed_banks.columns = ['лицензия','дата']

In [536]:
closed_banks['default'] = 1

# Макро показатели

In [496]:
macro = pd.read_csv('macro_1.csv',sep=',', header = 0,  encoding="cp1251")

In [497]:
macro['Date'] = pd.to_datetime(macro['Date'])

In [499]:
macro.columns = ['drop','дата', 'M2', 'M2_SA', 'INF_SA', 'NX', 'NX_log', 'Nx_growth', 'usd_rub', 'usd_rub_std',\
                 'micex', 'micex_std', 'miacr', 'miacr_std', 'miacr_amount', 'net_foreign_assets',\
                 'net_gov_debt', 'other_fin_debt', 'retail_debt', 'stocks_capital', 'i_retail_credit', 'i_retail_debit',\
                 'i_retail_spread']

In [501]:
macro.drop(['drop'], inplace = True, axis = 1)

In [502]:
macro.head(5)

,дата,M2,M2_SA,INF_SA,NX,NX_log,Nx_growth,usd_rub,usd_rub_std,micex,...,miacr_std,miacr_amount,net_foreign_assets,net_gov_debt,other_fin_debt,retail_debt,stocks_capital,i_retail_credit,i_retail_debit,i_retail_spread
0,2009-12-01,14967.6,14477.677786,0.020000,13107,9.480902,0,29.60000,0.180000,1433.648,...,1.181512,2643995,14099099,-6769343,516858,3959248,6004665,17.1,6.0,11.1
1,2010-01-01,15267.6,14477.677786,0.020000,15835,9.669978,2728,29.83873,0.374754,1433.648,...,0.724981,2643995,13942674,-5174038,564379,3932605,6219422,17.1,6.0,11.1
2,2010-02-01,14904.1,14875.445322,0.027475,14855,9.606092,-980,30.15797,0.177248,1351.718,...,0.330918,2643995,14168103,-5439869,529987,3913803,6279331,17.2,6.2,11.0
3,2010-03-01,15236.4,15247.121207,0.024986,14556,9.585759,-299,29.55940,0.238446,1410.415,...,0.359785,2643995,14039665,-5008998,508111,3898961,6283624,17.1,6.0,11.1
4,2010-04-01,15639.4,15570.868119,0.021233,13789,9.531626,-767,29.19324,0.158573,1478.745,...,0.246044,2643995,13815998,-4916261,507036,3910081,6347867,17.1,6.1,11.0


# Капитал банков

In [447]:
capital = pd.read_csv('capital_1.csv',sep=',', header = 0,  encoding="cp1251")

In [450]:
capital.columns = ['drop', 'date', 'bank', 'license', 'capital', 'msk_spb']
capital.drop(['drop'], inplace = True, axis = 1)

In [451]:
capital.dropna(axis=0, how='any', inplace = True)
capital[['license']] = capital[['license']].astype(int)
capital['date'] = pd.to_datetime(capital['date'])

In [452]:
capital.columns = ['дата', 'банк', 'лицензия', 'capital', 'msk_spb']

In [746]:
capital.tail(5)

,дата,банк,лицензия,capital,msk_spb
74869,2017-11-01,Элплат,3535,18995.0,0
75434,2017-12-01,Элплат,3535,18995.0,0
74277,2017-10-01,Р-Инкас,3536,102845.0,1
74849,2017-11-01,Р-Инкас,3536,97707.0,1
75415,2017-12-01,Р-Инкас,3536,90689.0,1


# Join всего в один датафрейм

In [455]:
bank_frame[['лицензия']] = bank_frame[['лицензия']].astype(int)
bank_frame['дата'] = pd.to_datetime(bank_frame['дата'])

In [471]:
capital.sort_values(by = ['лицензия', 'дата'], inplace = True)
bank_frame.sort_values(by = ['лицензия', 'дата'], inplace = True)
liquidity.sort_values(by = ['лицензия', 'дата'], inplace = True)

In [458]:
df = bank_frame.set_index(['лицензия', 'дата']).join(capital.set_index(['лицензия', 'дата']), how = 'inner')

In [477]:
df_1 = df.join(liquidity.set_index(['лицензия', 'дата']), how = 'left')

In [493]:
df_1['ИД_1800'] = df_1['ИД_1800'].fillna(0).astype(int)
df_1['N3'] = df_1['N3'].fillna(0).astype(int)

ValueError: invalid literal for int() with base 10: '1 372,32'

In [503]:
df_2 = df.join(macro.set_index(['дата']), how = 'inner')

In [512]:
df_3 = df_2.join(liquidity.set_index(['лицензия', 'дата']), how = 'left')

In [563]:
# Очистка данных
closed_banks['лицензия'] = closed_banks['лицензия'].str.replace('-К', '')
closed_banks['лицензия'] = closed_banks['лицензия'].str.replace('-Д', '')
closed_banks['лицензия'] = closed_banks['лицензия'].str.replace('-Р', '')
closed_banks['лицензия'] = closed_banks['лицензия'].str.replace('-K', '')
closed_banks['лицензия'] = closed_banks['лицензия'].str.replace('-к', '')
closed_banks = closed_banks.iloc[ :(closed_banks.shape[0] - 2), :]
closed_banks[['лицензия']] = closed_banks[['лицензия']].astype(int)
closed_banks['дата'] = pd.to_datetime(closed_banks['дата'])

In [578]:
closed_banks.sort_values(by = ['лицензия', 'дата'], inplace = True)

C:\Users\imaksimov002\AppData\Local\Continuum\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [673]:
df_4 = df_3.join(closed_banks.set_index(['лицензия', 'дата']), how = 'left')

In [626]:
df_4.shape

(74633, 54)

In [674]:
df_4['default'] = df_4['default'].fillna(0)
df_4.drop(['Лицензия_банк'], inplace = True, axis = 1)

In [593]:
df_4.to_csv('final_data_3' + '.csv',sep=',',header=True, index=True)

In [591]:
df_4.head(5)

ИД_10 ИД_1000   ИД_110   ИД_120 ИД_130 ИД_140 ИД_160  \
лицензия дата                                                                 
0        2010-02-01  423 017      27  112 770   60 000      0      0      0   
         2010-03-01  498 411      75  172 628   90 000      0      0      0   
         2010-04-01  571 220      54  211 860   90 000      0      0      0   
         2010-05-01  523 027      41  159 970   90 000      0      0      0   
         2010-06-01  473 713      31  131 782  135 000      0      0      0   

                    ИД_260 ИД_30   ИД_300   ...    net_foreign_assets  \
лицензия дата                               ...                         
0        2010-02-01      0   102  225 000   ...              14168103   
         2010-03-01      0   569  225 000   ...              14039665   
         2010-04-01    997   651  227 287   ...              13815998   
         2010-05-01    481   677  227 130   ...              14067700   
         2010-06-01    485   644  191 306   ...              14454661   

                    net_gov_debt other_fin_debt retail_debt stocks_capital  \
лицензия дата                                                                
0        2010-02-01     -5439869         529987     3913803        6279331   
         2010-03-01     -5008998         508111     3898961        6283624   
         2010-04-01     -4916261         507036     3910081        6347867   
         2010-05-01     -5051224         507117     3955282        6383875   
         2010-06-01     -4878647         496365     4014144        6356903   

                    i_retail_credit i_retail_debit i_retail_spread  N3  \
лицензия дата                                                            
0        2010-02-01            17.2            6.2            11.0 NaN   
         2010-03-01            17.1            6.0            11.1 NaN   
         2010-04-01            17.1            6.1            11.0 NaN   
         2010-05-01            17.1            6.0            11.1 NaN   
         2010-06-01            17.2            5.8            11.4 NaN   

                     default  
лицензия дата                 
0        2010-02-01      0.0  
         2010-03-01      0.0  
         2010-04-01      0.0  
         2010-05-01      0.0  
         2010-06-01      0.0  

[5 rows x 53 columns]

In [676]:
# очистка данных:

gg = lambda x: x.replace('?','-') # ? -> -
pp = lambda y: y.replace(' ', '') # убратб пробелы
cc = lambda z: z.replace(',','.') # , -> .

item = ['10', '1000','110','120','130', '140','160', '260', '30', '300', '360','40', '400', '50', '500', '60', '70', '1100', '200','1300', '1400', '1500', '401', '1800', '1700','1600']
for i in item:
    a = "ИД_" + i
    df_4[a] = df_4[a].astype(str)
    df_4[a] = df_4[a].apply(gg)
    df_4[a] = df_4[a].apply(pp)
    df_4[a] = df_4[a].apply(cc)

In [677]:
# часть данных оказалась в долях, а не в %: исправим это
df_4['ИД_1000'] = df_4['ИД_1000'].astype(float) + (df_4['ИД_1000'].astype(float) < 0)*df_4['ИД_1000'].astype(float)*99
df_4['ИД_1100'] = df_4['ИД_1100'].astype(float) + (df_4['ИД_1100'].astype(float) < 0)*df_4['ИД_1100'].astype(float)*99

In [679]:
# перевести в числовой тип
item = ['10', '1000','110','120','130', '140','160', '260', '30', '300', '360','40', '400', '50', '500', '60', '70', '1100', '200','1300', '1400', '1500', '401', '1800', '1700','1600']
for i in item:
    a = "ИД_" + i
    df_4[a] = df_4[a].astype(float)

In [651]:
df_4.to_csv('final_data_5' + '.csv',sep=',',header=True, index=True)

# Обработка пропусков (NaN)

In [680]:
df_5 = df_4

In [691]:
df_5.dropna(thresh = 16, subset = ['ИД_10', 'ИД_1000','ИД_110','ИД_120','ИД_130', 'ИД_140','ИД_160', 'ИД_260',\
                                  'ИД_30', 'ИД_300', 'ИД_360','ИД_40', 'ИД_400', 'ИД_50', 'ИД_500', 'ИД_60',\
                                  'ИД_70', 'ИД_1100'],
           inplace = True)

In [731]:
col = df_5.columns

'ИД_10'

In [704]:
# Сохраним данные о названиях банков
bank_names = df_5.iloc[:, 27]
bank_names.to_csv('bank_names' + '.csv',sep=',',header=True, index=True)

In [705]:
# уберу ненужные столбцы
df_5.drop(['лиц_дат', 'банк'], inplace = True, axis = 1)

In [721]:
# Трюк, чтобы индексы стали столбцами
df_5.to_csv('final_data_6' + '.csv',sep=',',header=True, index=True)
df_6 = pd.read_csv('final_data_6.csv',sep=',', header = 0,  encoding="cp1251")

# Отдельно запишем индексы
lic = df_6['лицензия']
dat = df_6['дата']

In [726]:
# Заполним пропуски медианой
from sklearn.preprocessing import Imputer
imr = Imputer(missing_values = 'NaN', strategy = 'median', axis = 0)
imr.fit(df_5)
df_imp = df_5
df_imp = imr.transform(df_5.values)

In [735]:
# Добавим индексы
df_imp = pd.DataFrame(df_imp, columns = col)
df_imp['лицензия'] = lic
df_imp['дата'] = dat
df_imp = df_imp.set_index(['лицензия', 'дата'])
df_imp.head(5)

ИД_10  ИД_1000    ИД_110    ИД_120  ИД_130  ИД_140  \
лицензия дата                                                                
0        2010-02-01  423017.0     27.0  112770.0   60000.0     0.0     0.0   
         2010-03-01  498411.0     75.0  172628.0   90000.0     0.0     0.0   
         2010-04-01  571220.0     54.0  211860.0   90000.0     0.0     0.0   
         2010-05-01  523027.0     41.0  159970.0   90000.0     0.0     0.0   
         2010-06-01  473713.0     31.0  131782.0  135000.0     0.0     0.0   

                     ИД_160  ИД_260  ИД_30    ИД_300   ...     \
лицензия дата                                          ...      
0        2010-02-01     0.0     0.0  102.0  225000.0   ...      
         2010-03-01     0.0     0.0  569.0  225000.0   ...      
         2010-04-01     0.0   997.0  651.0  227287.0   ...      
         2010-05-01     0.0   481.0  677.0  227130.0   ...      
         2010-06-01     0.0   485.0  644.0  191306.0   ...      

                     net_foreign_assets  net_gov_debt  other_fin_debt  \
лицензия дата                                                           
0        2010-02-01          14168103.0    -5439869.0        529987.0   
         2010-03-01          14039665.0    -5008998.0        508111.0   
         2010-04-01          13815998.0    -4916261.0        507036.0   
         2010-05-01          14067700.0    -5051224.0        507117.0   
         2010-06-01          14454661.0    -4878647.0        496365.0   

                     retail_debt  stocks_capital  i_retail_credit  \
лицензия дата                                                       
0        2010-02-01    3913803.0       6279331.0             17.2   
         2010-03-01    3898961.0       6283624.0             17.1   
         2010-04-01    3910081.0       6347867.0             17.1   
         2010-05-01    3955282.0       6383875.0             17.1   
         2010-06-01    4014144.0       6356903.0             17.2   

                     i_retail_debit  i_retail_spread     N3  default  
лицензия дата                                                         
0        2010-02-01             6.2             11.0  92.27      0.0  
         2010-03-01             6.0             11.1  92.27      0.0  
         2010-04-01             6.1             11.0  92.27      0.0  
         2010-05-01             6.0             11.1  92.27      0.0  
         2010-06-01             5.8             11.4  92.27      0.0  

[5 rows x 51 columns]

# Переименуем названия столбцов

In [736]:
df_imp.columns

Index(['ИД_10', 'ИД_1000', 'ИД_110', 'ИД_120', 'ИД_130', 'ИД_140', 'ИД_160',
       'ИД_260', 'ИД_30', 'ИД_300', 'ИД_360', 'ИД_40', 'ИД_400', 'ИД_50',
       'ИД_500', 'ИД_60', 'ИД_70', 'ИД_1100', 'ИД_200', 'ИД_1300', 'ИД_1400',
       'ИД_1500', 'ИД_401', 'ИД_1800', 'ИД_1700', 'ИД_1600', 'capital',
       'msk_spb', 'M2', 'M2_SA', 'INF_SA', 'NX', 'NX_log', 'Nx_growth',
       'usd_rub', 'usd_rub_std', 'micex', 'micex_std', 'miacr', 'miacr_std',
       'miacr_amount', 'net_foreign_assets', 'net_gov_debt', 'other_fin_debt',
       'retail_debt', 'stocks_capital', 'i_retail_credit', 'i_retail_debit',
       'i_retail_spread', 'N3', 'default'],
      dtype='object')

In [753]:
columns = ['net_assets', 'ROA', 'liquid', 'ibl', 'stocks', 'bond', 'oth_cap', 'sunk_retail_credit',\
          'NI', 'organization_credit', 'sunk_organization_credit', 'credit_portf', 'drop', 'sunk_credit_portf',\
          'organization_deposit', 'retail_deposit', 'security_tot', 'ROE', 'retail_credit', 'reserv_credit_perc',\
          'zalog_credit_perc', 'foreign_na_fr', 'retail_deposit_fr', 'N3', 'N2', 'N1', 'capital',\
           'msk_spb', 'M2', 'M2_SA', 'INF_SA', 'NX', 'NX_log', 'Nx_growth',\
           'usd_rub', 'usd_rub_std', 'micex', 'micex_std', 'miacr', 'miacr_std',\
           'miacr_amount', 'net_foreign_assets', 'net_gov_debt', 'other_fin_debt',\
           'retail_debt', 'stocks_capital', 'i_retail_credit', 'i_retail_debit',\
           'i_retail_spread', 'N3.1', 'default']

In [754]:
df_imp.columns = columns
df_imp.drop('drop', axis = 1, inplace = True)
df_imp.to_csv('FINAL_imp' + '.csv',sep=',',header=True, index=True)

In [758]:
# В столбце N3_early показатель Н3 до 2015-01-01, а в N3 - после. Надо их объединить

df_imp_1 = pd.read_csv('FINAL_imp.csv',sep=',', header = 0,  encoding="cp1251")
df_imp_1['N3'] = df_imp_1['N3']*(df_imp_1['дата'] > '2015-01-01') + df_imp_1['N3.1']*(df_imp_1['дата'] <= '2015-01-01')
df_imp_1.drop(['N3.1'], axis = 1, inplace = True)

In [759]:
# количество уникальных банков
len(np.unique(lic))

1049

In [760]:
df_imp_1.head(5)

,лицензия,дата,net_assets,ROA,liquid,ibl,stocks,bond,oth_cap,sunk_retail_credit,...,miacr_amount,net_foreign_assets,net_gov_debt,other_fin_debt,retail_debt,stocks_capital,i_retail_credit,i_retail_debit,i_retail_spread,default
0,0,2010-02-01,423017.0,27.0,112770.0,60000.0,0.0,0.0,0.0,0.0,...,2643995.0,14168103.0,-5439869.0,529987.0,3913803.0,6279331.0,17.2,6.2,11.0,0.0
1,0,2010-03-01,498411.0,75.0,172628.0,90000.0,0.0,0.0,0.0,0.0,...,2643995.0,14039665.0,-5008998.0,508111.0,3898961.0,6283624.0,17.1,6.0,11.1,0.0
2,0,2010-04-01,571220.0,54.0,211860.0,90000.0,0.0,0.0,0.0,997.0,...,2643995.0,13815998.0,-4916261.0,507036.0,3910081.0,6347867.0,17.1,6.1,11.0,0.0
3,0,2010-05-01,523027.0,41.0,159970.0,90000.0,0.0,0.0,0.0,481.0,...,2643995.0,14067700.0,-5051224.0,507117.0,3955282.0,6383875.0,17.1,6.0,11.1,0.0
4,0,2010-06-01,473713.0,31.0,131782.0,135000.0,0.0,0.0,0.0,485.0,...,2643995.0,14454661.0,-4878647.0,496365.0,4014144.0,6356903.0,17.2,5.8,11.4,0.0


In [761]:
df_imp_1.to_csv('FINAL_OK_imp' + '.csv',sep=',',header=True, index=True)